In [2]:
import rdkit
from rdkit import Chem
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import mytools
import pickle

In [3]:
process_pd = pd.read_csv('processed_data.csv')

# 处理蛋白质seq

In [4]:
seq_word_dict = mytools.build_word_dict(process_pd['seq'], 3)

# 保存word_dict
with open('seq_word_dict.pkl', 'wb') as f:
    pickle.dump(seq_word_dict, f)

处理中:: 100%|██████████| 7989/7989 [00:00<00:00, 14834.31it/s]


In [5]:
# 加载词库

file_path = 'seq_word_dict.pkl'
with open(file_path, 'rb') as f:
    word_dict = pickle.load(f)

In [6]:
embedding = nn.Embedding(len(word_dict), 100)
input = mytools.split_sequence(process_pd['seq'][0], 3, word_dict)
input = torch.tensor(input, dtype=torch.long)

In [9]:
input.shape

torch.Size([374])

In [76]:
embedding.weight.shape

torch.Size([8517, 100])

# 处理分子smiles2vec

# [smiles2vec教程](https://github.com/samoturk/mol2vec/blob/master/examples/01%20-%20Basic%20concepts%20of%20Mol2vec.ipynb)

In [1]:
from gensim.test.utils import common_texts
from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg

In [ ]:
# model = Word2Vec(sentences=common_texts, vector_size=10, window=5, min_count=1, workers=4)
# display(model.wv['computer'])
# display(model.wv.most_similar('computer', topn=3))

In [8]:
model = word2vec.Word2Vec.load('smi2vec_model.pkl')
display(len(model.wv.key_to_index.keys()))

21003

In [9]:
aa_smiles = process_pd['smiles']
aa_mols = [Chem.MolFromSmiles(smi) for smi in aa_smiles]
sentences = [mol2alt_sentence(mol, 1) for mol in aa_mols]
# IdentifierTable(sentence, [aa_mols[0]]*len(sentence), [sentence]*len(sentence), 5, 1)

In [51]:
# 更新词汇表，允许更新现有词汇表
model.build_vocab(sentences, update=True, min_count=1)

# 对新添加的词汇进行训练
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

# 保存更新后的模型
model.save('updated_smi2vec_model.pkl')